In [3]:
import pandas as pd
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import time
from IPython.display import clear_output
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from matplotlib.pylab import rcParams
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
rcParams['figure.figsize'] = 80,50
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.metrics import accuracy_score

In [64]:
#Preprocess training set
data=pd.read_csv('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/celeba/labels.csv')

labelsTrain = data["\timg_name\tgender\tsmiling"].str.split(pat="\t", n=-1, expand=True)
labelsTrain.drop(columns =[0,3], inplace = True)
labelsTrain.columns = ["img_name"  , "gender"]
labelsTrain = labelsTrain.astype({'gender': 'int32'})
labelsTrain = labelsTrain.astype({'img_name': 'string'})
labelsTrain["gender"] = labelsTrain["gender"].replace(-1, 0)
#Preprocess testing set
data=pd.read_csv('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/celeba_test/labels.csv')

labelsTest = data["\timg_name\tgender\tsmiling"].str.split(pat="\t", n=-1, expand=True)
labelsTest.drop(columns =[0,3], inplace = True)
labelsTest.columns = ["img_name"  , "gender"]
labelsTest = labelsTest.astype({'gender': 'int32'})
labelsTest = labelsTest.astype({'img_name': 'string'})
labelsTest["gender"] = labelsTest["gender"].replace(-1, 0)

In [65]:
#Load training set################################
imagesTrain = np.zeros((5000, 150, 150,3))

for i in range(0,5000):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/celeba/img/' + labelsTrain["img_name"][i])
    #image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image = cv.resize(image, dsize=(150 , 150), interpolation=cv.INTER_CUBIC)
    imagesTrain[i] = image
    
# imagesTrain = imagesTrain.reshape(5000,38804)    
# imagesTrain = pd.DataFrame(imagesTrain) 


#Load Testing set################################
imagesTest = np.zeros((1000, 150, 150,3))

for i in range(0,1000):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/celeba_test/img/' + labelsTest["img_name"][i])
    #image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image = cv.resize(image, dsize=(150 , 150), interpolation=cv.INTER_CUBIC)
    imagesTest[i] = image
    
# imagesTest = imagesTest.reshape(1000,38804)   

# imagesTest = pd.DataFrame(imagesTest) 

In [134]:
#scale coversion
scaler = MinMaxScaler()

imagesTrain_scaled = scaler.fit_transform(imagesTrain)
imagesTest_scaled = scaler.transform(imagesTest)

In [135]:
#PCA conversion
pca = PCA(n_components = 500)

imagesTrain_pca = pca.fit_transform(imagesTrain_scaled)
imagesTest_pca = pca.transform(imagesTest_scaled)

imagesTrain_pca = pd.DataFrame(imagesTrain_pca) 
imagesTest_pca = pd.DataFrame(imagesTest_pca) 

In [136]:
pca.explained_variance_ratio_.sum()

0.9586665346256535

In [137]:
x_train = pd.DataFrame(imagesTrain_pca) 
x_test = pd.DataFrame(imagesTest_pca)
x_train = pd.concat([labelsTrain,x_train],axis=1, join='inner')

In [151]:
#logistic regression#######
model = LogisticRegression(C=0.012742749857031334, max_iter=50, solver='sag')
model.fit(x_train.iloc[:,2:], labelsTrain['gender'])
model.score(x_test,labelsTest['gender'])

C:\Users\X99S5\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.897

In [146]:
hyper_params = {
    'max_iter' : [25,50,100],
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'penalty' : ['l1','l2','elasticnet','none'],
    'C': np.logspace(-4,4,20)
    
    
}

In [147]:
model = LogisticRegression()
grid=GridSearchCV(model , param_grid=hyper_params , cv=10 , n_jobs=-1 , verbose = True)

In [148]:
grid.fit(x_train.iloc[:,2:],labelsTrain['gender'])

Fitting 10 folds for each of 1200 candidates, totalling 12000 fits


C:\Users\X99S5\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
5400 fits failed out of a total of 12000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
600 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\X99S5\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\X99S5\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py", line 1091, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\X99S5\AppData\Local\Program

GridSearchCV(cv=10, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04]),
                         'max_iter': [25, 50, 100],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                         'solver': ['lbfgs', 'newton-cg', 'liblinear', 'sag',
                                    'saga']},
             verbose=True)

In [152]:
grid.best_estimator_

LogisticRegression(C=0.012742749857031334, max_iter=50, solver='sag')

In [150]:
grid.best_score_

0.8906000000000001

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
print("num of gpus available: ",len(tf.config.experimental.list_physical_devices('GPU')))
from keras.utils import normalize,to_categorical
from keras.preprocessing.image import ImageDataGenerator
tf.config.run_functions_eagerly(True)
#needed for tf to run

num of gpus available:  1


In [66]:
labelsTrain = labelsTrain['gender']
labelsTest = labelsTest['gender']

imagesTrain = normalize(imagesTrain,axis=1)
imagesTest = normalize(imagesTest,axis=1)

labelsTrain = to_categorical(labelsTrain)
labelsTest = to_categorical(labelsTest)

In [67]:
datagen_train = ImageDataGenerator()
#needed to save memory

datagen_Test = ImageDataGenerator()

In [68]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add( Dense(128,activation='relu') )
model.add( Dense(64,activation='relu') )
model.add( Dense(2,activation='softmax') )

In [69]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [71]:
history = model.fit( 
    datagen_train.flow(imagesTrain, labelsTrain, batch_size=100),
    epochs=15,
    validation_data=datagen_Test.flow(imagesTest, labelsTest, batch_size=100),
)

Epoch 1/15
50/50 [==============================] - 7s 123ms/step - loss: 0.1176 - accuracy: 0.9526 - val_loss: 0.1739 - val_accuracy: 0.9390
Epoch 2/15
50/50 [==============================] - 6s 123ms/step - loss: 0.1048 - accuracy: 0.9560 - val_loss: 0.3645 - val_accuracy: 0.8850
Epoch 3/15
50/50 [==============================] - 6s 122ms/step - loss: 0.0890 - accuracy: 0.9668 - val_loss: 0.2446 - val_accuracy: 0.9240
Epoch 4/15
50/50 [==============================] - 6s 122ms/step - loss: 0.0527 - accuracy: 0.9804 - val_loss: 0.2264 - val_accuracy: 0.9310
Epoch 5/15
50/50 [==============================] - 6s 123ms/step - loss: 0.0785 - accuracy: 0.9712 - val_loss: 0.2232 - val_accuracy: 0.9370
Epoch 6/15
50/50 [==============================] - 6s 122ms/step - loss: 0.0431 - accuracy: 0.9840 - val_loss: 0.2282 - val_accuracy: 0.9310
Epoch 7/15
50/50 [==============================] - 6s 123ms/step - loss: 0.0302 - accuracy: 0.9888 - val_loss: 0.2839 - val_accuracy: 0.9330
Epoch 